# **Medical Tools Detection**



.

In [ ]:
from google.colab import drive
drive.mount("/content/mydrive")

Drive already mounted at /content/mydrive; to attempt to forcibly remount, call drive.mount("/content/mydrive", force_remount=True).


In [ ]:
import os
%cd /content/mydrive/MyDrive/project/surgical_tools_detection/yolov9
HOME = os.getcwd()
print(HOME)

/content/mydrive/MyDrive/project/surgical_tools_detection/yolov9
/content/mydrive/MyDrive/project/surgical_tools_detection/yolov9


# Install necessary dependencies

In [ ]:
!pip install -r requirements.txt

In [ ]:
!pip install gradio langchain

In [ ]:
!pip install -U langchain-openai

# - Object Detection Function

In [ ]:
import argparse
import os
import platform
import sys
from pathlib import Path

import torch
from models.common import DetectMultiBackend
from utils.dataloaders import IMG_FORMATS, VID_FORMATS, LoadImages, LoadScreenshots, LoadStreams
from utils.general import (LOGGER, Profile, check_file, check_img_size, check_imshow, check_requirements, colorstr, cv2,
                           increment_path, non_max_suppression, print_args, scale_boxes, strip_optimizer, xyxy2xywh)
from utils.plots import Annotator, colors, save_one_box
from utils.torch_utils import select_device, smart_inference_mode

@smart_inference_mode()
def run_inference(
        weights="yolo.pt",  # model path or triton URL
        source="",  # file/dir/URL/glob/screen/0(webcam)
        data="/content/mydrive/MyDrive/project/yolov9/data/custom.yaml",  # dataset.yaml path
        imgsz=(640, 640),  # inference size (height, width)
        conf_thres=0.25,  # confidence threshold
        iou_thres=0.45,  # NMS IOU threshold
        max_det=1000,  # maximum detections per image
        device='',  # cuda device, i.e. 0 or 0,1,2,3 or cpu
        view_img=False,  # show results
        save_txt=False,  # save results to *.txt
        save_conf=False,  # save confidences in --save-txt labels
        save_crop=False,  # save cropped prediction boxes
        nosave=False,  # do not save images/videos
        classes=None,  # filter by class: --class 0, or --class 0 2 3
        agnostic_nms=False,  # class-agnostic NMS
        augment=False,  # augmented inference
        visualize=False,  # visualize features
        update=False,  # update all models
        project="/content/mydrive/MyDrive/project/yolov9/runs/detect",  # save results to project/name
        name='exp',  # save results to project/name
        exist_ok=False,  # existing project/name ok, do not increment
        line_thickness=3,  # bounding box thickness (pixels)
        hide_labels=False,  # hide labels
        hide_conf=False,  # hide confidences
        half=False,  # use FP16 half-precision inference
        dnn=False,  # use OpenCV DNN for ONNX inference
        vid_stride=1,  # video frame-rate stride
            ):



    source = str(source)
    save_img = not nosave and not source.endswith('.txt')  # save inference images
    is_file = Path(source).suffix[1:] in (IMG_FORMATS + VID_FORMATS)
    is_url = source.lower().startswith(('rtsp://', 'rtmp://', 'http://', 'https://'))
    webcam = source.isnumeric() or source.endswith('.txt') or (is_url and not is_file)
    screenshot = source.lower().startswith('screen')
    detected_tools = []
    if is_url and is_file:
        source = check_file(source)  # download

    # Directories
    save_dir = increment_path(Path(project) / name, exist_ok=exist_ok)  # increment run
    (save_dir / 'labels' if save_txt else save_dir).mkdir(parents=True, exist_ok=True)  # make dir

    # Load model
    device = select_device(device)
    model = DetectMultiBackend(weights, device=device, dnn=dnn, data=data, fp16=half)
    stride, names, pt = model.stride, model.names, model.pt
    imgsz = check_img_size(imgsz, s=stride)  # check image size

    # Dataloader
    bs = 1  # batch_size
    if webcam:
        view_img = check_imshow(warn=True)
        dataset = LoadStreams(source, img_size=imgsz, stride=stride, auto=pt, vid_stride=vid_stride)
        bs = len(dataset)
    elif screenshot:
        dataset = LoadScreenshots(source, img_size=imgsz, stride=stride, auto=pt)
    else:
        dataset = LoadImages(source, img_size=imgsz, stride=stride, auto=pt, vid_stride=vid_stride)
    vid_path, vid_writer = [None] * bs, [None] * bs

    # Run inference
    model.warmup(imgsz=(1 if pt or model.triton else bs, 3, *imgsz))  # warmup
    seen, windows, dt = 0, [], (Profile(), Profile(), Profile())
    for path, im, im0s, vid_cap, s in dataset:
        with dt[0]:
            im = torch.from_numpy(im).to(model.device)
            im = im.half() if model.fp16 else im.float()  # uint8 to fp16/32
            im /= 255  # 0 - 255 to 0.0 - 1.0
            if len(im.shape) == 3:
                im = im[None]  # expand for batch dim

        # Inference
        with dt[1]:
            visualize = increment_path(save_dir / Path(path).stem, mkdir=True) if visualize else False
            pred = model(im, augment=augment, visualize=visualize)

        # NMS
        with dt[2]:
            pred = pred[0][1] if isinstance(pred[0], list) else pred[0]
            pred = non_max_suppression(pred, conf_thres, iou_thres, classes, agnostic_nms, max_det=max_det)

        # Second-stage classifier (optional)
        # pred = utils.general.apply_classifier(pred, classifier_model, im, im0s)

        # Process predictions
        for i, det in enumerate(pred):  # per image
            seen += 1
            if webcam:  # batch_size >= 1
                p, im0, frame = path[i], im0s[i].copy(), dataset.count
                s += f'{i}: '
            else:
                p, im0, frame = path, im0s.copy(), getattr(dataset, 'frame', 0)

            p = Path(p)  # to Path
            save_path = str(save_dir / p.name)  # im.jpg
            txt_path = str(save_dir / 'labels' / p.stem) + ('' if dataset.mode == 'image' else f'_{frame}')  # im.txt
            s += '%gx%g ' % im.shape[2:]  # print string
            gn = torch.tensor(im0.shape)[[1, 0, 1, 0]]  # normalization gain whwh
            imc = im0.copy() if save_crop else im0  # for save_crop
            annotator = Annotator(im0, line_width=line_thickness, example=str(names))
            if len(det):
                # Rescale boxes from img_size to im0 size
                det[:, :4] = scale_boxes(im.shape[2:], det[:, :4], im0.shape).round()

                # Print results
                for c in det[:, 5].unique():
                    n = (det[:, 5] == c).sum()  # detections per class
                    s += f"{n} {names[int(c)]}{'s' * (n > 1)}, "  # add to string
                    detected_tools.append(names[int(c)])

                # Write results
                for *xyxy, conf, cls in reversed(det):
                    if save_txt:  # Write to file
                        xywh = (xyxy2xywh(torch.tensor(xyxy).view(1, 4)) / gn).view(-1).tolist()  # normalized xywh
                        line = (cls, *xywh, conf) if save_conf else (cls, *xywh)  # label format
                        with open(f'{txt_path}.txt', 'a') as f:
                            f.write(('%g ' * len(line)).rstrip() % line + '\n')

                    if save_img or save_crop or view_img:  # Add bbox to image
                        c = int(cls)  # integer class
                        label = None if hide_labels else (names[c] if hide_conf else f'{names[c]} {conf:.2f}')
                        annotator.box_label(xyxy, label, color=colors(c, True))
                    if save_crop:
                        save_one_box(xyxy, imc, file=save_dir / 'crops' / names[c] / f'{p.stem}.jpg', BGR=True)

            # Stream results
            im0 = annotator.result()

            return im0, detected_tools




inference using the detect file

In [ ]:
import cv2
import time
def detect_object (source):
  global classes
  cv2.imwrite('input_img.jpg', source)
  time.sleep(2)

  source = 'input_img.jpg'
  weights = "/content/mydrive/MyDrive/project/surgical_tools_detection/yolov9/runs/train/exp3/weights/best.pt"

  col_infe, classes = run_inference(weights=weights, source=source)

  return col_infe

# - Load the OpenAI key into your **environment**

In [ ]:
os.environ["OPENAI_API_KEY"] ='sk-TEbRNi8lPk9t03I99yGIT3BlbkFJ9pJW1ZbgKFSCfWhxy'    # paste your own key

# - Run the chatbot function

In [ ]:
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain.chains import LLMChain

model_name = 'gpt-3.5-turbo'

llm = ChatOpenAI(model_name=model_name, temperature=0.4)

def getResponse (user_message):

  instrument_descriptions = {
    'Army_navy': "Army-navy retractors are handheld instruments used to hold back tissue or organs during surgical procedures. They are commonly used in orthopedic and abdominal surgeries. To sterilize, soak in a high-level disinfectant solution or autoclave at high pressure and temperature.",
    'Bulldog': "Bulldog clamps are small, spring-loaded clamps used to occlude blood vessels or hold tissue. They are often used in neurosurgery, cardiovascular surgery, and other procedures requiring temporary vessel occlusion. Sterilize by autoclaving or soaking in a sterilizing solution.",
    'Castroviejo': "Castroviejo needle holders are precision instruments used to grasp and manipulate fine needles during suturing. They are commonly used in ophthalmic and plastic surgeries. To sterilize, autoclave at high pressure and temperature or soak in a sterilizing solution.",
    'Forceps': "Forceps are versatile surgical instruments used for grasping, holding, or manipulating tissues, sutures, or other objects during procedures. They come in various shapes and sizes for different purposes. Sterilize by autoclaving, boiling, or soaking in a sterilizing solution.",
    'Frazier': "Frazier suction tips are narrow, fine-tipped suction instruments used to aspirate fluid or debris from surgical sites, particularly in neurosurgery and otolaryngology procedures. Sterilize by autoclaving, soaking in a sterilizing solution, or using ethylene oxide gas sterilization.",
    'Hemostat': "Hemostats, also known as artery forceps, are clamping instruments used to control bleeding by clamping blood vessels or tissue. They are essential in various surgical specialties. Sterilize by autoclaving, boiling, or soaking in a sterilizing solution.",
    'Iris': "Iris scissors are small, delicate scissors with short, sharp blades and fine tips. They are commonly used in ophthalmic surgeries and other procedures requiring precision cutting. Sterilize by autoclaving, soaking in a sterilizing solution, or using ethylene oxide gas sterilization.",
    'Mayo_metz': "Mayo-Metz scissors, also known as dissecting scissors, are versatile surgical scissors with robust blades and blunt tips. They are used for cutting tough tissues and sutures in various surgical specialties. Sterilize by autoclaving, boiling, or soaking in a sterilizing solution.",
    'Needle': "Surgical needles are sharp, slender instruments used for suturing tissues during surgical procedures. They come in various sizes, shapes, and configurations for different purposes. Sterilize by autoclaving, soaking in a sterilizing solution, or using ethylene oxide gas sterilization.",
    'Potts': "Potts scissors are long-handled scissors with curved blades and sharp, pointed tips. They are primarily used for dissecting and cutting delicate tissues in neurosurgery and other specialties. Sterilize by autoclaving, soaking in a sterilizing solution, or using ethylene oxide gas sterilization.",
    'Richardson': "Richardson retractors, also known as wound retractors, are handheld instruments used to hold back tissues or organs and maintain surgical exposure during procedures. They come in various sizes and shapes for different surgical sites. Sterilize by autoclaving or soaking in a sterilizing solution.",
    'Scalpel': "Scalpels are precision cutting instruments used for making incisions in tissues during surgical procedures. They consist of a handle and a sharp, replaceable blade. Sterilize by autoclaving, soaking in a sterilizing solution, or using ethylene oxide gas sterilization.",
    'Towel_clip': "Towel clips, also known as towel forceps or towel clamps, are spring-loaded clamps used to secure surgical drapes or towels to the patient's skin to maintain a sterile field. They are essential for infection control in surgery. Sterilize by autoclaving or soaking in a sterilizing solution.",
    'Weitlaner': "Weitlaner retractors are self-retaining, double-ended instruments used to hold back tissues and maintain surgical exposure during procedures. They come in various sizes and styles for different surgical specialties. Sterilize by autoclaving or soaking in a sterilizing solution.",
    'Yankauer': "Yankauer suction tips, also known as tonsil suction tips, are rigid suction instruments used to remove blood, saliva, or other fluids from the surgical site. They are commonly used in oral and throat surgeries. Sterilize by autoclaving, soaking in a sterilizing solution, or using ethylene oxide gas sterilization."
  }
   context_info = [instrument_descriptions[cls] for cls in classes]
   all_context = " ".join(context_info)
   all_tools = ['Army_navy', 'Bulldog', 'Castroviejo', 'Forceps', 'Frazier', 'Hemostat', 'Iris',
                'Mayo_metz', 'Needle', 'Potts', 'Richardson', 'Scalpel', 'Towel_clip', 'Weitlaner', 'Yankauer']
   conversation_template  = f"""
                 you are a medical surgical assistant and a conversational assistant that help doctors and medical practitionals in the surgical room.
                 you are provided with the full list of all surgical tools that are to be used and also the tools that are replaced inside the tool box after the operation.

                 you are to figure out the missing tools in tool box after the operation. The doctor will ask some questions based on this tools and you must use your own knowledge to answer those questions.
                 You are responsible for identifying any missing tools and informing the doctor. If asked whether any tools are missing, please respond with 'yes' or 'no' accordingly. If a tool is missing, kindly request the doctor to replace it.

               Information on the surgical tools
               Context: {all_context}

               List of the complete surgical tools:
                 {all_tools},

               List of the tools replaced inside tool box:
                 {classes}
               user message: {user_message}
             """

   #initialize the embedchain bot
   PROMPT = PromptTemplate(
         template=conversation_template,
         input_variables= ['all_context','all_tools','classes','user_message']
     )

   chain = LLMChain(llm=llm, prompt=PROMPT)

   response = chain.predict(all_context= all_context, all_tools=all_tools, classes=classes, user_message = user_message)

  return response

# - Design the Gradio interface UI

In [ ]:
import gradio as gr
import numpy as np
import tensorflow as tf
from PIL import Image, ImageDraw


In [ ]:

# Create Gradio interface
image_input = gr.Image(label="Upload Image")
# print(image_input)
object_detection_output = gr.Image(label="Object Detection")
chat_input = gr.Textbox(lines=2, label="Ask a question")
chat_output = gr.Textbox(label="Chatbot Response")

title = "Surgical Tools Detection"
title1 = "Chatbot"
description = "Upload an image, detect objects, and ask questions about the detected objects."

detection_interface = gr.Interface(
    fn=detect_object,
    inputs=image_input,
    outputs=object_detection_output,
    title=title,
    description=description,
    allow_flagging='never'  # Disable flagging for simplicity
)

chat_interface = gr.Interface(
    fn=getResponse,
    inputs=chat_input,
    outputs=chat_output,
    title=title1,
    description="Ask questions about the detected objects.",
    allow_flagging='never'  # Disable flagging for simplicity
)

demo = gr.TabbedInterface([detection_interface, chat_interface], ["Surgical tool detection", "Chat with data"])

# - Lunch the Application User Interface

In [ ]:
demo.launch(share=True, debug = True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://2d90a50fab3a1abf22.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
